# Формирования таблицы словаря 'Marks' для базы данных.

### 1. Подготовка

In [1]:

import time
import json
import datetime

import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine


### 2. Формирование переменных для подключения

In [ ]:

TOKEN = 'Bearer '
API = '/api/marks/getall'
URL = 'https://tenderplan.ru' + API
headers = {
    'Authorization': TOKEN,
    'Accept': 'application/json'
}

engine_pet = create_engine('mysql+mysqlconnector://'':''@''/tender_project')


### 3. Подключение к API tenderplan

In [3]:

response_marks = rq.get(
    URL,
    headers = headers
).json()


### 4. Формируем функцию для преобразования UNIX-времени в правильный формат datetime

In [4]:

def f_date_fr_time(date_):
    '''Принимает UNIX-время и возвращате дату в формате datetime'''
    return datetime.datetime.fromtimestamp(date_ // 1000)


### 5. Формирование DataFrame 'marks' и преобразовываем в нужный вид

Формируем DataFrame с columns:
- _id - уникальный ключ марки;
- name - наименованеи марки;
- isDeleted - удаленА ли марка из базы;
- createDateTime - дата заведения марки (формат UNXI-время);
- updateDateTime - дата изменения марки (формат UNXI-время).

Для преобразования unix-времени в атрибуте updateDateTime были удалены значения nan в данном атрибуте, применена функция 'f_date_fr_time' для присвоения нового типа данных.
Также был добавлен новый атрибут 'true_name' где были изменены наименования марок.

In [5]:

df_marks = pd.DataFrame(
    response_marks,
    columns = [
        '_id',
        'name',
        'isDeleted',
        'createDateTime',
        'updateDateTime'
    ]
).set_index('_id').loc[lambda x: ~x['isDeleted']]

df_marks = df_marks.assign(
    **{
        'createDateTime': df_marks['createDateTime'].map(f_date_fr_time),
        'updateDateTime': (
            df_marks['updateDateTime']
            .dropna()
            .astype(np.int64)
            .map(f_date_fr_time)
        ),
        'name': df_marks['name'].map(lambda x: x.lower()),
        'true_name': df_marks.loc[
            [
                '63ca2494b2cb7552afeec78b',
                '63ca249dd606039bd7f2b40e',
                '65efe93047d03ddb11ee5953',
                '63cea8faa4e9171a4c97c4f8',
                '68467b2f7e6f0306f565c4df',
                '68467b455b73299890484d87'
            ],
            ['name']
        ].map(lambda x: x.lower().split()[0])
    }
).fillna({'true_name': 'прочее'})


### 6. Загружаем данные во временную таблицу 'tmp_dict_marks'

In [6]:

(
    df_marks
    .reset_index()
    .to_sql(
        'tmp_dict_marks', 
        con = engine_pet, 
        index = False, 
        if_exists = 'replace'
    )
)


-1

In [7]:

print(df_marks.head())


                                      name  isDeleted      createDateTime  \
_id                                                                         
63c8d71004a28500ff75d0aa      44 участвуем      False 2023-01-19 12:37:20   
63c8d71004a28500ff75d0ab  посмотреть итоги      False 2023-01-19 12:37:20   
63c8f53e3e7552dfcf3e9741         нп никита      False 2023-01-19 14:46:06   
63c8f7dd63fd9c162e85aacf       стас запрос      False 2023-01-19 14:57:17   
63c8fdd804a28500ff75f6a8          44 минус      False 2023-01-19 15:22:48   

                              updateDateTime true_name  
_id                                                     
63c8d71004a28500ff75d0aa 2024-07-19 13:33:14    прочее  
63c8d71004a28500ff75d0ab 2024-02-19 11:46:13    прочее  
63c8f53e3e7552dfcf3e9741 2025-05-20 12:19:13    прочее  
63c8f7dd63fd9c162e85aacf 2024-02-19 09:41:37    прочее  
63c8fdd804a28500ff75f6a8 2024-02-19 09:40:35    прочее  
